In [1]:
import pandas as pd
import numpy as np 
import datetime
import math
import gc
import requests
gc.collect()

from tqdm import tqdm

# Load Processed Data

In [2]:
PATH = './output/processed.csv'
df = pd.read_csv(PATH)

## Challenges

There are two main challenges when weighting an EPA model.  

One, when teams get up 30 points, they will stop running efficient plays and just run clock. There needs to be some sort of penalty weight for plays when the score is out of hand. I'll try and arrive to this penalty empirically.  

Two, in the Ron Yurko et. al. paper I'm referencing, they assign weighting penalties if the next score is 4 or 5 drives in the future. Essentially, in that case, the current drive isn't deterministic when it comes to expected points. Again, I'll try to arrive to this penalty empirically.  

I'll start with the second challenge, and I'm going to approach it slightly differently. First, I'm going to take a small subset of the data and try to predict drives till next score. This should be near zero when in the opponent's redzone, and might max out (just a guess, 1.5) around a team's own 25. With multinomial logit, I can get a probability of zero, one, two, etc drives till next score. Then, using the rest of the data, I can group by drives till next score and predict probabilities of each type of score that way.

In [3]:
gb = df.groupby(['game_id','drive_id'])['down'].count().reset_index()
# gb = gb.groupby(['down'])['drive_id'].count()
# gb = gb.sort_values(ascending=False)
# gb = gb.reset_index()
# gb = gb.rename(columns={'drive_id':'play_count'})
_max = gb.down.max()
print(_max)
print(len(gb))

36
275017


In [4]:
# just for fun/validation, wanted to look at 
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

x = pd.Series(gb.down, name="play_count")
fig = plt.figure(figsize=[15,9])
# ax1 = sns.kdeplot(x, bw=0.14, label="play_count")
ax = sns.distplot(x, bins=np.arange(0,_max), kde=False, norm_hist=True)
ax.set(xlabel='play_count', ylabel='percentage of drives')
ax.set_title('Distribution of Play Count on 300,000 CFB Drives')
plt.show()

fig.savefig("./plots/play_counts.png")

<Figure size 1500x900 with 1 Axes>

In [28]:
# df[df['col_name'].str.contains(r'^(?=.*apple)(?=.*banana)')]

# base = r'^{}'
# expr = '(?=.*{})'
# words = ['apple', 'banana', 'cat']  # example
# base.format(''.join(expr.format(w) for w in words))